# Quantum Chemistry with Paddle Quantum's qchem
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

qchem, which builds on top of Paddle Quantum, is designed to be a toolkit for quantum chemistry research in quantum computing era. It provides high level APIs for researchers who are interested in leveraging their current quantum chemsitry calculation with quantum computing power. And it also allows experts to write customized code. qchem is currently under active development, feel free to join us by opening issues or pull requests.

## Calculating ground state energy of a molecule
qchem provides various tools for you to do quantum chemistry calculation. Currently, qchem module supports the following molecular wave function ansatz:
* Hardware Efficient ansatz[<sup>1</sup>](#refer-1),
* Hartree Fock ansatz[<sup>2</sup>](#refer-2),
* Unitary Coupled Cluster singles and doubles (UCCSD) ansatz[<sup>3</sup>](#refer-3).

Let's start from example and try to calculate the ground state energy of hydrogen molecule. First, we need to import qchem.

In [1]:
import paddle_quantum as pq
from paddle_quantum import qchem as pq_qchem
import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.INFO)

/home/zl/miniconda3/envs/pq/lib/python3.8/site-packages/openfermion/hamiltonians/hartree_fock.py:11: DeprecationWarning: Please use `OptimizeResult` from the `scipy.optimize` namespace, the `scipy.optimize.optimize` namespace is deprecated.
  from scipy.optimize.optimize import OptimizeResult
/home/zl/miniconda3/envs/pq/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/home/zl/miniconda3/envs/pq/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in 

Then, we will build a `Molecule` class for a molecule based on properites such as geometry, charge, chemical basis set.

In [2]:
# `driver` is used to calculate various molecular integrals.
driver = pq_qchem.PySCFDriver()

# Build a Molecule class based on its properties, note, the length unit is Angstrom.
mol = pq_qchem.Molecule(
    geometry=[("H", [0.0, 0.0, 0.0]), ("H", [0.0, 0.0, 0.74])],
    basis="sto-3g",
    multiplicity=1,   
    driver=driver
)

Next, we need to choose an wavefunction ansatz for our hydrogen molecule. Let's choose `HardwareEfficient` as the ansatz. 

In [3]:
# Build a quantum circuit using HardwareEfficient ansatz.
mol.build()
n_qubits = mol.num_qubits
depth = 2
cir = pq_qchem.HardwareEfficient(n_qubits, depth)

INFO:root:
#######################################
Molecule
#######################################
INFO:root:H2
INFO:root:Geometry:
INFO:root:H 0.00000, 0.00000, 0.00000
H 0.00000, 0.00000, 0.74000
INFO:root:Charge: 0
INFO:root:Multiplicity: 1
INFO:root:Unit: Angstrom
INFO:root:
#######################################
SCF Calculation (Classical)
#######################################
INFO:root:Basis: sto-3g


converged SCF energy = -1.11675930739643


INFO:root:SCF energy: -1.11676.


Finally, we can call `GroundStateSolver` and use PaddlePaddle's optimizer to update parameters in the ansatz.

In [4]:
# We choose `Adam` optimizer in PaddlePaddle
from paddle.optimizer import Adam

solver = pq_qchem.GroundStateSolver(Adam, num_iterations=100, tol=1e-5, save_every=10)
e, psi = solver.solve(cir, mol, learning_rate=0.5)

INFO:root:
#######################################
VQE (Ground State)
#######################################
INFO:root:Number of qubits: 4
INFO:root:Ansatz: HardwareEfficient
INFO:root:Optimizer: Adam
INFO:root:	learning_rate: 0.5
INFO:root:
Optimization:
INFO:root:	Itr 0, loss=-0.47434.
INFO:root:	Itr 10, loss=-1.05781.
INFO:root:	Itr 20, loss=-1.10282.
INFO:root:	Itr 30, loss=-1.11235.
INFO:root:	Itr 40, loss=-1.11737.
INFO:root:Optimization converged after 46 iterations.
INFO:root:The final loss = -1.11891.


---
## References

[1] Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." [Nature 549.7671 (2017): 242-246.](https://www.nature.com/articles/nature23879)

[2] Arute, Frank, et al. "Hartree-Fock on a superconducting qubit quantum computer." [Science 369.6507 (2020): 1084-1089.](https://www.science.org/doi/10.1126/science.abb9811)

[3] Abhinav, Aspuru-Guzik, et al. "A Quantum Computing View on Unitary Coupled Cluster Theory" (https://arxiv.org/abs/2109.15176)